In [1]:
import requests
import xml.etree.ElementTree as ET
import csv
import os

# Function to extract URLs from an XML sitemap
def extract_urls(sitemap_url):
    response = requests.get(sitemap_url)
    response.raise_for_status()  # Ensure request was successful
    root = ET.fromstring(response.content)
    
    # Extract URLs
    urls = [elem.text for elem in root.findall('.//{http://www.sitemaps.org/schemas/sitemap/0.9}loc')]
    return urls

# Main sitemap URL
main_sitemap = "https://points-media.com/sitemap_index.xml"

# Output CSV file
csv_filename = "points_media_urls.csv"
file_exists = os.path.isfile(csv_filename)

# Open CSV file for appending
with open(csv_filename, "a", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    
    # Write header if file is new
    if not file_exists:
        writer.writerow(["URL"])  # Add additional columns if needed

    # Get all nested sitemap URLs
    sitemap_urls = extract_urls(main_sitemap)

    # Process each sitemap
    for sitemap in sitemap_urls:
        print(f"Processing: {sitemap}")
        try:
            article_urls = extract_urls(sitemap)
            for url in article_urls:
                writer.writerow([url])  # Append each URL
        except Exception as e:
            print(f"Error processing {sitemap}: {e}")

print(f"Scraping complete. Data saved in '{csv_filename}'.")

Processing: https://points-media.com/post-sitemap.xml
Processing: https://points-media.com/post-sitemap2.xml
Processing: https://points-media.com/post-sitemap3.xml
Processing: https://points-media.com/post-sitemap4.xml
Processing: https://points-media.com/post-sitemap5.xml
Processing: https://points-media.com/post-sitemap6.xml
Processing: https://points-media.com/page-sitemap.xml
Processing: https://points-media.com/category-sitemap.xml
Processing: https://points-media.com/post_tag-sitemap.xml
Processing: https://points-media.com/post_tag-sitemap2.xml
Processing: https://points-media.com/post_tag-sitemap3.xml
Processing: https://points-media.com/post_tag-sitemap4.xml
Processing: https://points-media.com/post_tag-sitemap5.xml
Processing: https://points-media.com/post_tag-sitemap6.xml
Processing: https://points-media.com/post_tag-sitemap7.xml
Processing: https://points-media.com/post_tag-sitemap8.xml
Processing: https://points-media.com/post_tag-sitemap9.xml
Processing: https://points-me

In [2]:
import requests
import csv
import os
import time
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from datetime import datetime

# Input file containing URLs
input_file = "points_media_urls.csv"
output_file = "scraped_articles.csv"
error_file = "scraping_errors.csv"

# Load URLs from the CSV
with open(input_file, "r", encoding="utf-8") as f:
    urls = [line.strip() for line in f.readlines()][1:]  # Skip header

# Check if output file exists to determine if we need to write a header
file_exists = os.path.isfile(output_file)
error_file_exists = os.path.isfile(error_file)

# Open CSV files for appending results
with open(output_file, "a", newline="", encoding="utf-8") as f_out, open(error_file, "a", newline="", encoding="utf-8") as f_err:
    writer = csv.writer(f_out)
    error_writer = csv.writer(f_err)

    # Write headers if files are new
    if not file_exists:
        writer.writerow(["ID", "Scrape_Time", "Publication_Time", "URL", "Source", "Title", "Section", "Content"])
    if not error_file_exists:
        error_writer.writerow(["URL", "Error_Message"])

    total_urls = len(urls)
    
    for index, url in enumerate(urls, start=1):
        print(f"Processing {index}/{total_urls}: {url}")

        try:
            # Fetch the webpage
            response = requests.get(url, timeout=10)
            response.raise_for_status()  # Raise an error for bad responses

            # Parse HTML
            soup = BeautifulSoup(response.text, "html.parser")

            # Extract title
            title_tag = soup.find("h1", class_="entry-title")
            title = title_tag.text.strip() if title_tag else "N/A"

            # Extract publication time
            pub_time_tag = soup.find("time", class_="entry-date updated td-module-date")
            publication_time = pub_time_tag["datetime"] if pub_time_tag else "N/A"

            # Extract content
            content_tag = soup.find("div", class_="td-post-content td-pb-padding-side")
            content = content_tag.get_text(" ", strip=True) if content_tag else "N/A"

            # Generate ID based on order encountered
            scrape_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            source = urlparse(url).netloc
            section = "中国新闻"

            # Write the result to CSV
            writer.writerow([index, scrape_time, publication_time, url, source, title, section, content])

            print(f"✓ Successfully scraped: {title[:50]}...")

        except Exception as e:
            print(f"✗ Error scraping {url}: {e}")
            error_writer.writerow([url, str(e)])

        # Small delay to avoid overloading the server
        time.sleep(1)

print(f"\nScraping complete. Results saved in '{output_file}'. Errors logged in '{error_file}'.")


Processing 1/16215: https://points-media.com/uncategorized/%e7%be%8e%e7%89%b9%e5%8b%a4%e5%b1%80%ef%bc%9a%e4%b8%ad%e5%9c%8b%e9%bb%91%e5%ae%a2%e7%ab%8a%e5%8f%96%e7%be%8e%e5%9c%8b%e9%98%b2%e7%96%ab%e6%95%91%e6%bf%9f%e9%87%91/
✓ Successfully scraped: 美特勤局：中國黑客竊取美國防疫救濟金...
Processing 2/16215: https://points-media.com/uncategorized/%e6%b5%81%e4%ba%a1%e6%be%b3%e6%b4%b2-%e9%87%8d%e6%8b%be%e6%b3%95%e5%be%8b%e5%b0%88%e6%a5%ad-%e5%be%8b%e5%b8%ab%e6%a8%93%e5%af%a6%e7%bf%92-%e8%a8%b1%e6%99%ba%e5%b3%af%e8%bf%91%e6%b3%81%ef%bc%9a/
✓ Successfully scraped: 流亡澳洲 重拾法律專業 律師樓實習 許智峯近況：無穿無爛！...
Processing 3/16215: https://points-media.com/%e6%be%b3%e6%b4%b2-%e4%ba%9e%e5%a4%aa/%e6%be%b3%e6%b4%b2%e5%85%ad%e5%90%8d%e8%b7%a8%e9%bb%a8%e6%b4%be%e6%9c%83%e8%ad%b0%e5%93%a1%e8%a8%aa%e5%8f%b0-%e4%b8%ad%e5%85%b1%e8%bd%9f%e7%8e%a9%e7%81%ab/
✓ Successfully scraped: 澳洲六名跨黨派會議員訪台 中共轟玩火...
Processing 4/16215: https://points-media.com/uncategorized/%e6%be%b3%e7%be%8e%e6%8a%97%e4%b8%ad%e5%85%b1%e5%8d%b0%e5%a4%aa%e6%93%b4%e5%b

Processing 25/16215: https://points-media.com/%e6%be%b3%e6%b4%b2-%e4%ba%9e%e5%a4%aa/%e6%9d%8e%e5%ae%b6%e8%b6%85%e5%ae%a3%e5%b8%83%e6%af%8b%e9%a0%88%e6%8e%83%e3%80%8c%e5%ae%89%e5%bf%83%e5%87%ba%e8%a1%8c%e3%80%8d%e5%8f%96%e6%b6%88%e9%bb%83%e7%a2%bc/
✓ Successfully scraped: 李家超宣布毋須掃「安心出行」取消黃碼...
Processing 26/16215: https://points-media.com/%e9%a6%99%e6%b8%af/%e3%80%8a%e7%ab%8b%e5%a0%b4%e6%96%b0%e8%81%9e%e3%80%8b%e9%8d%be%e6%b2%9b%e6%ac%8a%e6%89%b9%e5%87%86%e4%bf%9d%e9%87%8b%e5%80%99%e5%af%a9-%e8%be%af%e6%96%b9%e8%ab%8b%e7%94%b3%e6%b0%b8%e4%b9%85%e7%b5%82/
✓ Successfully scraped: 《立場新聞》鍾沛權批准保釋候審  辯方請申永久終止聆訊...
Processing 27/16215: https://points-media.com/uncategorized/%e5%8a%a0%e5%af%86%e8%b2%a8%e5%b9%a3%e5%a4%a7%e7%8e%8b%e5%b7%b4%e5%93%88%e9%a6%ac%e8%a2%ab%e6%8d%95-%e5%b0%87%e5%bc%95%e6%b8%a1%e5%9b%9e%e7%be%8e%e5%8f%97%e5%af%a9/
✓ Successfully scraped: 加密貨幣大王巴哈馬被捕 將引渡回美受審...
Processing 28/16215: https://points-media.com/%e6%9c%80%e6%96%b0/%e6%a7%8d%e6%93%8a%e6%a1%88%e4%b8%89%e5%90%8d%e7

✓ Successfully scraped: 又搶藥！中國防疫放寬 澳洲中國代購搶購必理痛...
Processing 51/16215: https://points-media.com/%e5%8a%a0%e6%8b%bf%e5%a4%a7-%e5%8c%97%e7%be%8e/%e5%8a%a0%e6%8b%bf%e5%a4%a7%e5%a4%ae%e8%a1%8c%e5%b9%b4%e5%85%a77%e5%ba%a6%e5%8a%a0%e6%81%af-%e7%8f%be%e6%96%b7%e4%be%9b%e8%b3%a3%e6%a8%93%e6%bd%ae-%e6%a5%ad%e4%b8%bb%e7%82%ba%e4%be%9b%e6%a8%93/
✓ Successfully scraped: 加拿大央行年內7度加息 現斷供賣樓潮 業主為供樓打2份工 每周工作80小時...
Processing 52/16215: https://points-media.com/%e6%be%b3%e6%b4%b2-%e4%ba%9e%e5%a4%aa/%e7%b6%ad%e5%b7%9e%e9%80%81%e5%a4%a7%e7%a6%ae-%e8%81%96%e8%aa%95%e9%99%a4%e5%a4%95%e7%81%ab%e8%bb%8a%e5%b7%b4%e5%a3%ab%e5%85%8d%e8%b2%bb%e6%90%ad/
✓ Successfully scraped: 維州送大禮 聖誕除夕火車巴士免費搭...
Processing 53/16215: https://points-media.com/%e9%a6%99%e6%b8%af/%e3%80%902019%e5%8f%8d%e4%bf%ae%e4%be%8b%e3%80%91%e6%b6%88%e6%81%af%ef%bc%9a6000%e5%90%8d%e6%9c%aa%e8%a2%ab%e6%aa%a2%e6%8e%a7%e8%80%85%e3%80%80%e5%a4%a7%e5%a4%9a%e3%80%8c%e5%91%8a%e4%b8%80%e6%ae%b5/
✓ Successfully scraped: 【2019反修例】消息：6,000名未被檢控者　大多「告一段落」..

✓ Successfully scraped: Elon Musk掛住玩Twitter失首富地位 全球億萬富豪排行榜 香港跑贏新加坡...
Processing 77/16215: https://points-media.com/%e4%b8%ad%e5%9c%8b/%e6%92%90%e3%80%8c%e7%99%bd%e7%b4%99%e9%9d%a9%e5%91%bd%e3%80%8d%e9%81%ad%e7%ae%97%e5%b8%b3-%e5%8d%97%e9%96%8b%e5%a4%a7%e5%ad%b8%e5%a5%b3%e8%ac%9b%e5%b8%ab%e8%a2%ab%e5%bc%b7%e9%80%81%e7%b2%be%e7%a5%9e/
✓ Successfully scraped: 撐「白紙革命」遭算帳! 南開大學女講師被強送精神病院 後認胡言亂語...
Processing 78/16215: https://points-media.com/%e5%8a%a0%e6%8b%bf%e5%a4%a7-%e5%8c%97%e7%be%8e/iphone-14%e7%a2%b0%e6%92%9e%e5%81%b5%e6%b8%ac%e7%b3%bb%e7%b5%b1%e3%80%8c%e9%80%a3%e7%92%b0%e7%82%92%e8%bb%8a%e3%80%8d-%e5%88%86%e4%b8%8d%e6%b8%85%e6%88%b6%e5%a4%96%e6%b4%bb%e5%8b%95-%e8%a7%b8/
✓ Successfully scraped: iPhone 14碰撞偵測系統「連環炒車」  分不清戶外活動 觸發直昇機搜山...
Processing 79/16215: https://points-media.com/%e5%8a%a0%e6%8b%bf%e5%a4%a7-%e5%8c%97%e7%be%8e/%e5%8a%a0%e6%8b%bf%e5%a4%a7%e4%bd%8f%e5%ae%85%e7%a7%9f%e9%87%91%e7%a0%b4%e6%ad%b7%e5%8f%b2%e7%b4%80%e9%8c%84-%e9%81%a0%e8%b6%85%e9%80%9a%e8%84%b9%e6%b0%b4%e5%

✗ Error scraping https://points-media.com/%e4%b8%ad%e5%9c%8b/%e7%be%8e%e5%9c%8b%e6%8a%95%e8%b3%87550%e5%84%84%e7%be%8e%e5%85%83all-in%e9%9d%9e%e6%b4%b2-%e6%8b%9c%e7%99%bb%e7%a8%b1%e5%87%b8%e9%a1%af%e5%85%b1%e5%90%8c%e6%88%90%e5%8a%9f-%e6%9a%97%e8%ab%b7%e8%88%87/: HTTPSConnectionPool(host='points-media.com', port=443): Max retries exceeded with url: /%E4%B8%AD%E5%9C%8B/%E7%BE%8E%E5%9C%8B%E6%8A%95%E8%B3%87550%E5%84%84%E7%BE%8E%E5%85%83all-in%E9%9D%9E%E6%B4%B2-%E6%8B%9C%E7%99%BB%E7%A8%B1%E5%87%B8%E9%A1%AF%E5%85%B1%E5%90%8C%E6%88%90%E5%8A%9F-%E6%9A%97%E8%AB%B7%E8%88%87/ (Caused by SSLError(SSLZeroReturnError(6, 'TLS/SSL connection has been closed (EOF) (_ssl.c:1131)')))
Processing 103/16215: https://points-media.com/%e6%9c%80%e6%96%b0/%e9%a6%99%e6%b8%af%e8%bf%913%e5%80%8b%e6%9c%88%e5%a4%b1%e6%a5%ad%e7%8e%873-7-%e4%bb%8d%e6%9c%8913-9%e8%90%ac%e4%ba%ba%e5%86%87%e5%b7%a5%e9%96%8b-%e5%8b%9e%e7%a6%8f%e5%b1%80%ef%bc%9a%e6%94%be%e5%af%ac/
✓ Successfully scraped: 香港近3個月失業率3.7% 仍有13.9萬人冇工開 勞福局：放寬防疫

...

✓ Successfully scraped: 標籤: 香港藝術發展局...
Processing 15899/16215: https://points-media.com/tag/%e9%a6%99%e6%b8%af%e8%a1%8c%e6%94%bf%e9%95%b7%e5%ae%98%e6%9d%8e%e5%ae%b6%e8%b6%85/
✓ Successfully scraped: 標籤: 香港行政長官李家超...
Processing 15900/16215: https://points-media.com/tag/%e9%a6%99%e6%b8%af%e8%a1%97%e9%a0%ad/
✓ Successfully scraped: 標籤: 香港街頭...
Processing 15901/16215: https://points-media.com/tag/%e9%a6%99%e6%b8%af%e8%a8%80%e8%aa%9e%e6%b2%bb%e7%99%82%e5%b8%ab%e7%b8%bd%e5%b7%a5%e6%9c%83/
✓ Successfully scraped: 標籤: 香港言語治療師總工會...
Processing 15902/16215: https://points-media.com/tag/%e9%a6%99%e6%b8%af%e8%a8%98%e5%8d%94/
✓ Successfully scraped: 標籤: 香港記協...
Processing 15903/16215: https://points-media.com/tag/%e9%a6%99%e6%b8%af%e8%a8%98%e8%80%85%e5%8d%94%e6%9c%83/
✓ Successfully scraped: 標籤: 香港記者協會...
Processing 15904/16215: https://points-media.com/tag/%e9%a6%99%e6%b8%af%e8%a8%98%e9%8c%84%e8%80%85/
✓ Successfully scraped: 標籤: 香港記錄者...
Processing 15905/16215: https://points-media.com/tag/%e9%a6

✓ Successfully scraped: 標籤: 馬雲...
Processing 15961/16215: https://points-media.com/tag/%e9%a6%ac%e9%be%8d/
✓ Successfully scraped: 標籤: 馬龍...
Processing 15962/16215: https://points-media.com/tag/%e9%a6%ae%e5%be%b7%e8%90%8a%e6%81%a9/
✓ Successfully scraped: 標籤: 馮德萊恩...
Processing 15963/16215: https://points-media.com/tag/%e9%a6%ae%e6%99%9e%e4%b9%be/
✓ Successfully scraped: 標籤: 馮晞乾...
Processing 15964/16215: https://points-media.com/tag/%e9%a6%ae%e7%9d%8e%e4%b9%be/
✓ Successfully scraped: 標籤: 馮睎乾...
Processing 15965/16215: https://points-media.com/tag/%e9%a6%ae%e7%9d%8e%e4%b9%be%e5%8d%81%e4%b8%89%e7%b6%ad%e5%ba%a6/
✓ Successfully scraped: 標籤: 馮睎乾十三維度...
Processing 15966/16215: https://points-media.com/tag/%e9%a6%ae%e7%a5%bf%e5%be%b7/
✓ Successfully scraped: 標籤: 馮祿德...
Processing 15967/16215: https://points-media.com/tag/%e9%a6%ae%e7%a8%8b%e6%b7%91%e5%84%80/
✓ Successfully scraped: 標籤: 馮程淑儀...
Processing 15968/16215: https://points-media.com/tag/%e9%a7%90%e4%b8%ad%e5%9c%8b%e8%be%a6%e4%ba%8

✓ Successfully scraped: 標籤: 高強度排練...
Processing 16026/16215: https://points-media.com/tag/%e9%ab%98%e5%bf%97%e5%87%b1/
✓ Successfully scraped: 標籤: 高志凱...
Processing 16027/16215: https://points-media.com/tag/%e9%ab%98%e5%bf%97%e6%b4%bb/
✓ Successfully scraped: 標籤: 高志活...
Processing 16028/16215: https://points-media.com/tag/%e9%ab%98%e5%bf%97%e6%b4%bb%e5%85%ac%e9%96%8b%e4%bf%a1/
✓ Successfully scraped: 標籤: 高志活公開信...
Processing 16029/16215: https://points-media.com/tag/%e9%ab%98%e6%81%af/
✓ Successfully scraped: 標籤: 高息...
Processing 16030/16215: https://points-media.com/tag/%e9%ab%98%e6%89%8d%e9%80%9a/
✓ Successfully scraped: 標籤: 高才通...
Processing 16031/16215: https://points-media.com/tag/%e9%ab%98%e6%8a%80%e8%83%bd%e5%b7%a5%e4%ba%ba/
✓ Successfully scraped: 標籤: 高技能工人...
Processing 16032/16215: https://points-media.com/tag/%e9%ab%98%e6%8a%80%e8%a1%93%e5%b7%a5%e4%bd%9c/
✓ Successfully scraped: 標籤: 高技術工作...
Processing 16033/16215: https://points-media.com/tag/%e9%ab%98%e6%94%b6%e5%85%a5%e5%

Processing 16093/16215: https://points-media.com/tag/%e9%ba%97%e8%8f%af%e7%8f%a0%e5%af%b6%e5%ba%97/
✓ Successfully scraped: 標籤: 麗華珠寶店...
Processing 16094/16215: https://points-media.com/tag/%e9%ba%97%e9%83%bd%e9%81%8b%e6%b2%b3/
✓ Successfully scraped: 標籤: 麗都運河...
Processing 16095/16215: https://points-media.com/tag/%e9%ba%a5%e5%8d%a1%e9%8c%ab/
✓ Successfully scraped: 標籤: 麥卡錫...
Processing 16096/16215: https://points-media.com/tag/%e9%ba%a5%e5%8d%a1%e9%8d%9a/
✓ Successfully scraped: 標籤: 麥卡鍚...
Processing 16097/16215: https://points-media.com/tag/%e9%ba%a5%e5%98%89%e7%90%b3/
✓ Successfully scraped: 標籤: 麥嘉琳...
Processing 16098/16215: https://points-media.com/tag/%e9%ba%a5%e5%ba%b7%e5%a5%88%e7%88%be/
✓ Successfully scraped: 標籤: 麥康奈爾...
Processing 16099/16215: https://points-media.com/tag/%e9%ba%a5%e6%87%bf%e7%9d%bf/
✓ Successfully scraped: 標籤: 麥懿睿...
Processing 16100/16215: https://points-media.com/tag/%e9%ba%a5%e6%a8%b8%e6%80%9d/
✓ Successfully scraped: 標籤: 麥樸思...
Processing 16101/16215: 

✓ Successfully scraped: 標籤: 黃金簽證...
Processing 16161/16215: https://points-media.com/tag/%e9%bb%83%e9%8c%a6%e8%bc%9d/
✓ Successfully scraped: 標籤: 黃錦輝...
Processing 16162/16215: https://points-media.com/tag/%e9%bb%83%e9%9b%aa%e7%90%b4/
✓ Successfully scraped: 標籤: 黃雪琴...
Processing 16163/16215: https://points-media.com/tag/%e9%bb%8e%e5%b4%87%e6%81%a9/
✓ Successfully scraped: 標籤: 黎崇恩...
Processing 16164/16215: https://points-media.com/tag/%e9%bb%8e%e5%b7%b4%e5%ab%a9/
✓ Successfully scraped: 標籤: 黎巴嫩...
Processing 16165/16215: https://points-media.com/tag/%e9%bb%8e%e6%99%ba%e8%8b%b1/
✓ Successfully scraped: 標籤: 黎智英...
Processing 16166/16215: https://points-media.com/tag/%e9%bb%8e%e6%99%ba%e8%8b%b1%e5%9c%8b%e5%ae%89%e6%a1%88/
✓ Successfully scraped: 標籤: 黎智英國安案...
Processing 16167/16215: https://points-media.com/tag/%e9%bb%8e%e6%99%ba%e8%8b%b1%e6%a1%88/
✓ Successfully scraped: 標籤: 黎智英案...
Processing 16168/16215: https://points-media.com/tag/%e9%bb%8e%e6%99%ba%e8%8b%b1%e8%89%af%e5%bf%83%e7%8a%